In [1]:
# ============ 1. ПАРСЕР С REAL-TIME ДАННЫМИ ============
import nest_asyncio
nest_asyncio.apply()

import asyncio
import aiohttp
import json
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
from typing import List, Dict, Optional, Any, Union, Annotated
import logging
import hashlib
import re
import os
import warnings
warnings.filterwarnings('ignore')

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("  Библиотеки загружены и окружение настроено!")

# ============ 2. PYDANTIC МОДЕЛИ С ИСПРАВЛЕННОЙ ВАЛИДАЦИЕЙ ============
from pydantic import BaseModel, Field, field_validator, BeforeValidator, ConfigDict
from enum import Enum
from sentence_transformers import SentenceTransformer
import typing

# Функция-валидатор для age_restriction
def validate_age_restriction(value: Any) -> Optional[str]:
    """Преобразует age_restriction в правильный строковый формат"""
    if value is None:
        return None
    
    # Если это число
    if isinstance(value, (int, float)):
        if value == 0:
            return "0+"
        return f"{int(value)}+"
    
    # Если это строка
    if isinstance(value, str):
        value = value.strip()
        if not value:
            return None
        
        # Если строка содержит только цифры
        if value.isdigit():
            if value == "0":
                return "0+"
            return f"{value}+"
        
        # Если уже содержит "+" или другой формат
        if "+" in value or "лет" in value.lower() or "год" in value.lower():
            return value
        
        # Пытаемся извлечь число из строки
        match = re.search(r'\d+', value)
        if match:
            num = match.group()
            return f"{num}+"
        
        # Если ничего не нашли, возвращаем как есть
        return value
    
    # Для других типов преобразуем в строку
    return str(value)

# Аннотированный тип с валидатором
AgeRestrictionType = Annotated[Optional[str], BeforeValidator(validate_age_restriction)]

class EventCategory(str, Enum):
    """Категории событий (реальные из Kudago)"""
    CONCERT = "concert"
    THEATER = "theater"
    EXHIBITION = "exhibition"
    FESTIVAL = "festival"
    EDUCATION = "education"
    PARTY = "party"
    SPORT = "sport"
    QUEST = "quest"
    EXCURSION = "excursion"
    SHOW = "show"
    STANDUP = "standup"
    KIDS = "kids"
    FASHION = "fashion"
    GASTRONOMY = "gastronomy"
    CINEMA = "cinema"
    LECTURE = "lecture"
    MASTERCLASS = "masterclass"
    TOUR = "tour"
    OTHER = "other"

class PlaceModel(BaseModel):
    """Модель места с реальными данными"""
    model_config = ConfigDict(from_attributes=True)
    
    id: Optional[int] = None
    title: str = Field(..., description="Название места")
    address: Optional[str] = None
    subway: Optional[str] = None
    coords: Optional[Dict[str, float]] = None
    phone: Optional[str] = None
    site_url: Optional[str] = None
    city: Optional[str] = None
    is_closed: bool = False
    working_hours: Optional[Dict] = None
    
    @property
    def full_address(self) -> str:
        parts = []
        if self.title:
            parts.append(self.title)
        if self.address:
            parts.append(self.address)
        if self.subway:
            parts.append(f"м. {self.subway}")
        return ", ".join(parts)

class DateModel(BaseModel):
    """Модель даты события"""
    model_config = ConfigDict(from_attributes=True)
    
    start: int  # timestamp
    end: Optional[int] = None
    is_continuous: bool = False
    
    @property
    def start_dt(self) -> datetime:
        return datetime.fromtimestamp(self.start)
    
    @property
    def end_dt(self) -> Optional[datetime]:
        return datetime.fromtimestamp(self.end) if self.end else None
    
    @property
    def formatted(self) -> str:
        start_str = self.start_dt.strftime("%d.%m.%Y %H:%M")
        if self.end:
            end_str = self.end_dt.strftime("%d.%m.%Y %H:%M")
            if start_str[:10] == end_str[:10]:
                return f"{start_str} - {end_str[11:]}"
            return f"{start_str} - {end_str}"
        return start_str

class PriceModel(BaseModel):
    """Модель цены"""
    model_config = ConfigDict(from_attributes=True)
    
    is_free: bool = False
    min: Optional[float] = None
    max: Optional[float] = None
    currency: str = "RUB"
    description: Optional[str] = None
    
    @property
    def display(self) -> str:
        if self.is_free:
            return "Бесплатно"
        elif self.min and self.max:
            return f"{self.min:.0f} - {self.max:.0f} ₽"
        elif self.min:
            return f"от {self.min:.0f} ₽"
        return self.description or "Цена не указана"

class ImageModel(BaseModel):
    """Модель изображения"""
    model_config = ConfigDict(from_attributes=True)
    
    url: str
    thumbnail: Optional[str] = None
    source: Optional[Dict] = None

class TagModel(BaseModel):
    """Модель тега"""
    model_config = ConfigDict(from_attributes=True)
    
    name: str
    slug: Optional[str] = None
    id: Optional[int] = None

class ParticipantModel(BaseModel):
    """Модель участника"""
    model_config = ConfigDict(from_attributes=True)
    
    name: str
    role: Optional[str] = None
    images: Optional[List[Dict]] = None

class EventModel(BaseModel):
    """ПОЛНАЯ МОДЕЛЬ СОБЫТИЯ С РЕАЛЬНЫМИ ДАННЫМИ И ИСПРАВЛЕННОЙ ВАЛИДАЦИЕЙ"""
    model_config = ConfigDict(from_attributes=True, json_encoders={datetime: lambda dt: dt.isoformat()})
    
    id: int
    title: str
    short_title: Optional[str] = None
    description: Optional[str] = None
    slug: Optional[str] = None
    category: EventCategory
    tags: List[TagModel] = []
    dates: List[DateModel] = []
    publication_date: Optional[int] = None
    age_restriction: AgeRestrictionType = None  # Исправленный тип с валидацией
    place: Optional[PlaceModel] = None
    place_id: Optional[int] = None
    price: Optional[PriceModel] = None
    images: List[ImageModel] = []
    video: Optional[Dict] = None
    participants: List[ParticipantModel] = []
    url: str
    site_url: Optional[str] = None
    buy_url: Optional[str] = None
    favorites_count: int = 0
    comments_count: int = 0
    external_id: Optional[str] = None
    location: str = "msk"
    is_free: bool = False
    is_exclusive: bool = False
    is_editors_choice: bool = False
    is_approved: bool = True
    parsed_at: datetime = Field(default_factory=datetime.now)
    
    # Дополнительные валидаторы для полей
    @field_validator('price', mode='before')
    @classmethod
    def validate_price(cls, v):
        """Валидация и нормализация цены"""
        if v is None:
            return None
        if isinstance(v, dict):
            # Убедимся, что is_free корректно установлен
            if v.get('is_free') and ('min' in v or 'max' in v):
                # Если is_free=True, но есть цены - исправляем
                v = {**v, 'min': None, 'max': None}
            return v
        return v
    
    @field_validator('is_free', mode='before')
    @classmethod
    def validate_is_free(cls, v):
        """Валидация is_free"""
        if isinstance(v, bool):
            return v
        if isinstance(v, str):
            return v.lower() in ['true', '1', 'yes', 'да']
        return bool(v)
    
    @property
    def embedding_text(self) -> str:
        """Текст для создания эмбеддинга"""
        parts = [self.title]
        if self.description:
            parts.append(self.description[:500])
        if self.tags:
            parts.extend([tag.name for tag in self.tags[:5]])
        if self.place:
            if self.place.title:
                parts.append(self.place.title)
            if self.place.address:
                parts.append(self.place.address)
        parts.append(self.category.value)
        return " ".join(parts)
    
    @property
    def date_range_text(self) -> str:
        """Форматированный диапазон дат"""
        if not self.dates:
            return "Дата не указана"
        if len(self.dates) == 1:
            return self.dates[0].formatted
        dates_str = [date.formatted for date in self.dates[:3]]
        if len(self.dates) > 3:
            dates_str.append(f"... еще {len(self.dates)-3}")
        return "; ".join(dates_str)
    
    @property
    def normalized_age_restriction(self) -> str:
        """Нормализованное возрастное ограничение"""
        if not self.age_restriction:
            return "0+"
        return self.age_restriction

print("  Pydantic модели созданы с исправленной валидацией!")

# ============ 3. ИСПРАВЛЕННЫЙ ПАРСЕР KUDAGO API ============
class RealKudaGoParser:
    """Парсер реальных данных с Kudago API с исправленной обработкой"""
    
    def __init__(
        self,
        city: str = "msk",
        categories: Optional[List[str]] = None,
        max_events_per_category: int = 100,
        page_size: int = 100,
        timeout: int = 30,
        retries: int = 3,
        enable_enrichment: bool = True,
        days_ahead: int = 365
    ):
        self.base_url = "https://kudago.com/public-api/v1.4"
        self.city = city
        self.city_name = "Москва" if city == "msk" else "Санкт-Петербург"
        self.max_events_per_category = max_events_per_category
        self.page_size = min(page_size, 100)
        self.timeout = timeout
        self.retries = retries
        self.enable_enrichment = enable_enrichment
        self.days_ahead = days_ahead
        
        # Реальные категории, которые точно работают с Kudago API
        self.available_categories = {
            "concert": "Концерты",
            "theater": "Театр",
            "exhibition": "Выставки",
            "festival": "Фестивали",
            "education": "Образование",
            "party": "Вечеринки",
            "sport": "Спорт",
            "quest": "Квесты",
            "excursion": "Экскурсии",
            "show": "Шоу",
            "standup": "Стендап",
            "kids": "Детские",
            "fashion": "Мода",
            "gastronomy": "Гастрономия",
            "cinema": "Кино",
            "lecture": "Лекции",
            "masterclass": "Мастер-классы",
            "tour": "Туры",
            "yoga": "Йога",
            "performance": "Перформансы",
            "circus": "Цирк",
            "fair": "Ярмарки",
            "opera": "Опера",
            "ballet": "Балет"
        }
        
        # Надежные категории
        self.categories = categories or [
            "concert", "theater", "exhibition", "festival",
            "education", "party", "show", "kids",
            "cinema", "fashion", "gastronomy", "standup"
        ]
        
        # Проверяем категории
        for cat in self.categories:
            if cat not in self.available_categories:
                logger.warning(f"Категория '{cat}' может быть недоступна в API")
        
        logger.info(f"Парсер инициализирован для города: {self.city_name}")
        logger.info(f"Категорий: {len(self.categories)}")
        logger.info(f"Цель: {max_events_per_category} событий на категорию")
    
    def normalize_age_restriction(self, age_data: Any) -> Optional[str]:
        """Нормализует возрастное ограничение"""
        if age_data is None:
            return None
        
        # Если это число
        if isinstance(age_data, (int, float)):
            if age_data == 0:
                return "0+"
            return f"{int(age_data)}+"
        
        # Если это строка
        if isinstance(age_data, str):
            age_data = age_data.strip()
            if not age_data:
                return None
            
            # Если строка содержит только цифры
            if age_data.isdigit():
                if age_data == "0":
                    return "0+"
                return f"{age_data}+"
            
            # Если уже содержит "+" или другой формат
            if "+" in age_data or "лет" in age_data.lower() or "год" in age_data.lower():
                return age_data
            
            # Пытаемся извлечь число из строки
            match = re.search(r'\d+', age_data)
            if match:
                num = match.group()
                return f"{num}+"
            
            # Если ничего не нашли, возвращаем как есть
            return age_data
        
        # Для других типов преобразуем в строку
        return str(age_data)
    
    def normalize_price(self, price_data: Any) -> Dict:
        """Нормализует данные о цене"""
        if not price_data or not isinstance(price_data, dict):
            return {"is_free": False}
        
        normalized = price_data.copy()
        
        # Обработка is_free
        if "is_free" in normalized:
            is_free = normalized["is_free"]
            if isinstance(is_free, str):
                normalized["is_free"] = is_free.lower() in ['true', '1', 'yes', 'да']
            elif isinstance(is_free, int):
                normalized["is_free"] = bool(is_free)
        
        # Если is_free=True, очищаем min/max
        if normalized.get("is_free"):
            normalized["min"] = None
            normalized["max"] = None
        
        # Преобразование числовых значений
        for field in ['min', 'max']:
            if field in normalized and normalized[field] is not None:
                try:
                    normalized[field] = float(normalized[field])
                except (ValueError, TypeError):
                    normalized[field] = None
        
        return normalized
    
    async def fetch_with_retry(self, session: aiohttp.ClientSession, url: str, params: Dict) -> Optional[Dict]:
        """Запрос с повторными попытками"""
        for attempt in range(self.retries):
            try:
                async with session.get(
                    url,
                    params=params,
                    timeout=aiohttp.ClientTimeout(total=self.timeout)
                ) as response:
                    if response.status == 200:
                        return await response.json()
                    elif response.status == 429:
                        wait_time = 2 ** attempt
                        logger.warning(f"Rate limit, ждем {wait_time} сек...")
                        await asyncio.sleep(wait_time)
                        continue
                    elif response.status == 400:
                        logger.debug(f"Bad request: {response.status}")
                        return None
                    else:
                        logger.warning(f"HTTP {response.status} для {url}")
                        return None
            except asyncio.TimeoutError:
                logger.warning(f"Таймаут, попытка {attempt + 1}/{self.retries}")
                if attempt < self.retries - 1:
                    await asyncio.sleep(1)
                continue
            except Exception as e:
                logger.error(f"Ошибка запроса: {e}")
                return None
        return None
    
    async def fetch_events_page(self, session: aiohttp.ClientSession, category: str, page: int = 1) -> Optional[Dict]:
        """Загрузка страницы событий"""
        now = datetime.now()
        future_date = now + timedelta(days=self.days_ahead)
        
        params = {
            'categories': category,
            'location': self.city,
            'page_size': self.page_size,
            'page': page,
            'fields': 'id,title,description,dates,place,price,images,tags,age_restriction,'
                     'favorites_count,comments_count,site_url,is_free,participants,slug,short_title,'
                     'publication_date,location',
            'text_format': 'text',
            'order_by': '-dates',
            'actual_since': int(now.timestamp()),
            'actual_until': int(future_date.timestamp()),
        }
        
        url = f"{self.base_url}/events/"
        return await self.fetch_with_retry(session, url, params)
    
    async def fetch_place_details(self, session: aiohttp.ClientSession, place_id: int) -> Optional[Dict]:
        """Получение деталей места"""
        try:
            url = f"{self.base_url}/places/{place_id}/"
            async with session.get(
                url,
                timeout=aiohttp.ClientTimeout(total=15)
            ) as response:
                if response.status == 200:
                    return await response.json()
                return None
        except Exception:
            return None
    
    async def enrich_place_info(self, session: aiohttp.ClientSession, place_data: Dict) -> Dict:
        """Обогащение информации о месте"""
        if not place_data or not isinstance(place_data, dict):
            return {}
        
        place_id = place_data.get('id')
        if not place_id:
            return place_data
        
        try:
            place_details = await self.fetch_place_details(session, place_id)
            if place_details:
                for field in ['title', 'address', 'subway', 'coords', 'phone', 'site_url', 'city']:
                    if field in place_details:
                        place_data[field] = place_details[field]
        except Exception as e:
            logger.debug(f"Ошибка обогащения места {place_id}: {e}")
        
        return place_data
    
    def parse_event(self, raw_event: Dict, category_name: str) -> Optional[Dict]:
        """Парсинг сырого события в форматированный словарь"""
        try:
            # Проверяем обязательные поля
            event_id = raw_event.get('id')
            title = raw_event.get('title', '')
            
            if not event_id or not title:
                return None
            
            # Базовые данные
            event_data = {
                'id': event_id,
                'title': str(title),
                'short_title': raw_event.get('short_title'),
                'description': str(raw_event.get('description', ''))[:2000],
                'slug': raw_event.get('slug'),
                'category': category_name,
                'url': raw_event.get('site_url', f"https://kudago.com/{self.city}/event/{event_id}/"),
                'site_url': raw_event.get('site_url'),
                'is_free': raw_event.get('is_free', False),
                'age_restriction': self.normalize_age_restriction(raw_event.get('age_restriction')),  # Нормализованное значение
                'favorites_count': raw_event.get('favorites_count', 0),
                'comments_count': raw_event.get('comments_count', 0),
            }
            
            # Даты
            dates = []
            dates_text = []
            for date_info in raw_event.get('dates', []):
                if isinstance(date_info, dict) and date_info.get('start'):
                    start = date_info['start']
                    end = date_info.get('end')
                    
                    dates.append({
                        'start': start,
                        'end': end,
                        'is_continuous': date_info.get('is_continuous', False)
                    })
                    
                    try:
                        start_dt = datetime.fromtimestamp(start)
                        date_str = start_dt.strftime("%d.%m.%Y %H:%M")
                        
                        if end:
                            end_dt = datetime.fromtimestamp(end)
                            if start_dt.date() == end_dt.date():
                                date_str += f" - {end_dt.strftime('%H:%M')}"
                            else:
                                date_str += f" - {end_dt.strftime('%d.%m.%Y %H:%M')}"
                        
                        dates_text.append(date_str)
                    except:
                        continue
            
            event_data['dates'] = dates
            event_data['dates_text'] = dates_text
            
            # Место
            place_info = raw_event.get('place', {})
            if isinstance(place_info, dict):
                event_data['place'] = place_info
                if 'id' in place_info:
                    event_data['place_id'] = place_info['id']
                
                # Текст места
                place_parts = []
                if place_info.get('title'):
                    place_parts.append(str(place_info['title']))
                if place_info.get('address'):
                    place_parts.append(str(place_info['address']))
                if place_info.get('subway'):
                    place_parts.append(f"м. {place_info['subway']}")
                
                event_data['place_text'] = ", ".join(place_parts)[:300] if place_parts else ""
            
            # Цена
            price_info = raw_event.get('price', {})
            price_info = self.normalize_price(price_info)
            event_data['price'] = price_info
            
            price_text = "Цена не указана"
            if price_info.get('is_free'):
                price_text = "Бесплатно"
            elif price_info.get('min') is not None and price_info.get('max') is not None:
                price_text = f"{price_info['min']} - {price_info['max']} ₽"
            elif price_info.get('min') is not None:
                price_text = f"от {price_info['min']} ₽"
            elif price_info.get('description'):
                price_text = str(price_info['description'])[:100]
            
            event_data['price_text'] = price_text
            
            # Изображения
            images = []
            for img in raw_event.get('images', [])[:5]:
                if isinstance(img, dict):
                    images.append({
                        'url': str(img.get('image', '')),
                        'thumbnail': str(img.get('thumbnail', '')),
                        'source': img.get('source', {})
                    })
            
            event_data['images'] = images
            event_data['image_count'] = len(images)
            
            # Теги
            tags = []
            for tag in raw_event.get('tags', [])[:10]:
                if isinstance(tag, dict):
                    tags.append({
                        'name': str(tag.get('name', '')),
                        'slug': tag.get('slug'),
                        'id': tag.get('id')
                    })
                elif isinstance(tag, str):
                    tags.append({'name': str(tag)})
            
            event_data['tags'] = tags
            
            # Участники
            participants = []
            for participant in raw_event.get('participants', [])[:5]:
                if isinstance(participant, dict):
                    participants.append({
                        'name': str(participant.get('name', participant.get('title', ''))),
                        'role': participant.get('role'),
                        'images': participant.get('images', [])
                    })
            
            event_data['participants'] = participants
            
            # Дополнительные поля
            event_data['city'] = self.city
            event_data['city_name'] = self.city_name
            event_data['parsed_at'] = datetime.now().isoformat()
            
            # Публикационная дата
            if raw_event.get('publication_date'):
                event_data['publication_date'] = raw_event['publication_date']
            
            return event_data
            
        except Exception as e:
            logger.error(f"Ошибка парсинга события {raw_event.get('id')}: {e}")
            return None
    
    async def parse_category(self, category_code: str, category_name: str) -> List[Dict]:
        """Парсинг всей категории"""
        all_events = []
        page = 1
        max_pages = 20
        
        print(f"\n Парсинг категории: {category_name}")
        
        async with aiohttp.ClientSession() as session:
            while len(all_events) < self.max_events_per_category and page <= max_pages:
                logger.info(f"  Страница {page}...")
                
                data = await self.fetch_events_page(session, category_code, page)
                
                if not data or 'results' not in data:
                    logger.warning(f"  Нет данных на странице {page}")
                    break
                
                raw_events = data['results']
                if not raw_events:
                    logger.info(f"  Нет событий на странице {page}")
                    break
                
                # Парсим события
                parsed_events = []
                for raw_event in raw_events:
                    event_dict = self.parse_event(raw_event, category_name)
                    if event_dict:
                        # Обогащаем место, если включено
                        if self.enable_enrichment and event_dict.get('place'):
                            event_dict['place'] = await self.enrich_place_info(
                                session, event_dict['place']
                            )
                            # Обновляем текстовое представление места
                            if event_dict['place']:
                                place = event_dict['place']
                                place_parts = []
                                if place.get('title'):
                                    place_parts.append(str(place['title']))
                                if place.get('address'):
                                    place_parts.append(str(place['address']))
                                if place.get('subway'):
                                    place_parts.append(f"м. {place['subway']}")
                                
                                if place_parts:
                                    event_dict['place_text'] = ", ".join(place_parts)[:300]
                        
                        parsed_events.append(event_dict)
                
                logger.info(f"  Спаршено: {len(parsed_events)} событий")
                all_events.extend(parsed_events)
                
                # Проверяем, есть ли следующая страница
                next_url = data.get('next')
                if not next_url or len(raw_events) < self.page_size:
                    break
                
                page += 1
                await asyncio.sleep(1)  # Пауза между страницами
        
        print(f"   Категория '{category_name}': {len(all_events)} событий")
        return all_events
    
    async def parse_all_categories(self, max_concurrent: int = 3) -> Dict[str, List[Dict]]:
        """Парсинг всех категорий"""
        print(f"\n Начинаем парсинг города: {self.city_name}")
        print(f" Категорий: {len(self.categories)}")
        print(f" Цель: {self.max_events_per_category} событий на категорию")
        print("-" * 50)
        
        results = {}
        total_events = 0
        
        # Создаем задачи для каждой категории
        tasks = []
        for cat_code in self.categories:
            cat_name = self.available_categories.get(cat_code, cat_code)
            task = self.parse_category(cat_code, cat_name)
            tasks.append((cat_name, task))
        
        # Ограничиваем одновременные запросы
        semaphore = asyncio.Semaphore(max_concurrent)
        
        async def run_with_limit(cat_name, task):
            async with semaphore:
                try:
                    return cat_name, await task
                except Exception as e:
                    logger.error(f"Ошибка при парсинге категории {cat_name}: {e}")
                    return cat_name, []
        
        # Запускаем все задачи
        all_tasks = [run_with_limit(cat_name, task) for cat_name, task in tasks]
        results_list = await asyncio.gather(*all_tasks, return_exceptions=True)
        
        # Обрабатываем результаты
        for result in results_list:
            if isinstance(result, Exception):
                logger.error(f"Исключение при парсинге: {result}")
                continue
            
            cat_name, events = result
            results[cat_name] = events
            total_events += len(events)
        
        # Отчет
        print("\n" + "="*50)
        print(f"ПАРСИНГ ЗАВЕРШЕН!")
        print(f" Город: {self.city_name}")
        print(f" Всего событий: {total_events}")
        print("="*50)
        
        # Статистика по категориям
        print("\n СТАТИСТИКА ПО КАТЕГОРИЯМ:")
        successful_categories = 0
        
        for cat_name, events in results.items():
            if events:
                print(f"   {cat_name}: {len(events)} событий")
                successful_categories += 1
            else:
                print(f"   {cat_name}: 0 событий")
        
        print(f"\n Успешных категорий: {successful_categories}/{len(self.categories)}")
        
        return results

print("  Реальный парсер Kudago создан с исправленной обработкой данных!")

# ============ 4. ВЕКТОРНАЯ БАЗА ДАННЫХ С ИСПРАВЛЕННОЙ СХЕМОЙ ============
class RealEventVectorDatabase:
    """Векторная база данных для реальных событий с исправленной схемой"""
    
    def __init__(self, db_path: str = "real_events_vector.db", embedding_model: str = "all-MiniLM-L6-v2"):
        self.db_path = db_path
        
        try:
            # Загружаем модель для эмбеддингов
            logging.getLogger("sentence_transformers").setLevel(logging.WARNING)
            self.model = SentenceTransformer(embedding_model)
            self.embedding_dim = self.model.get_sentence_embedding_dimension()
        except Exception as e:
            logger.warning(f"Не удалось загрузить модель эмбеддингов: {e}")
            self.model = None
            self.embedding_dim = 384
        
        self.conn = sqlite3.connect(db_path)
        self._init_db()
        
        print(f" Векторная БД создана: {db_path}")
        print(f" Размерность эмбеддингов: {self.embedding_dim}")
    
    def _init_db(self):
        """Инициализация таблиц с исправленной схемой"""
        cursor = self.conn.cursor()
        
        # Основная таблица событий с исправленными типами
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS events (
            id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            description TEXT,
            category TEXT,
            dates_text TEXT,
            price_text TEXT,
            place_text TEXT,
            url TEXT,
            city TEXT,
            city_name TEXT,
            parsed_at TEXT,
            is_free INTEGER DEFAULT 0,
            age_restriction TEXT DEFAULT '0+',
            image_count INTEGER DEFAULT 0,
            embedding_text TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        
        # Таблица эмбеддингов
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS event_embeddings (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            event_id INTEGER NOT NULL,
            embedding BLOB,
            embedding_model TEXT DEFAULT 'all-MiniLM-L6-v2',
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (event_id) REFERENCES events (id) ON DELETE CASCADE,
            UNIQUE(event_id)
        )
        ''')
        
        # Таблица деталей (JSON данные)
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS event_details (
            id INTEGER PRIMARY KEY,
            event_id INTEGER NOT NULL,
            dates_json TEXT,
            price_json TEXT,
            place_json TEXT,
            tags_json TEXT,
            images_json TEXT,
            participants_json TEXT,
            raw_data_json TEXT,
            FOREIGN KEY (event_id) REFERENCES events (id) ON DELETE CASCADE,
            UNIQUE(event_id)
        )
        ''')
        
        # Таблица для отслеживания ошибок обработки
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS processing_errors (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            event_id INTEGER,
            error_type TEXT,
            error_message TEXT,
            raw_data TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        
        # Индексы для улучшения производительности
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_category ON events(category)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_city ON events(city)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_parsed_at ON events(parsed_at)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_is_free ON events(is_free)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_age_restriction ON events(age_restriction)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_event_embeddings_event_id ON event_embeddings(event_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_processing_errors_event_id ON processing_errors(event_id)')
        
        self.conn.commit()
    
    def create_embedding(self, text: str) -> np.ndarray:
        """Создание эмбеддинга для текста"""
        try:
            if not text or not text.strip():
                return np.zeros(self.embedding_dim, dtype=np.float32)
            
            if self.model:
                embedding = self.model.encode(text, show_progress_bar=False)
                return embedding.astype(np.float32)
            else:
                # Заглушка для тестирования
                return np.random.randn(self.embedding_dim).astype(np.float32)
        except Exception as e:
            logger.error(f"Ошибка создания эмбеддинга: {e}")
            return np.zeros(self.embedding_dim, dtype=np.float32)
    
    def save_event_with_error_handling(self, event: Dict) -> Dict:
        """Сохранение одного события с обработкой ошибок"""
        result = {
            'success': False,
            'event_id': event.get('id'),
            'errors': [],
            'warnings': []
        }
        
        try:
            cursor = self.conn.cursor()
            event_id = event['id']
            
            # Нормализация данных перед сохранением
            normalized_event = self._normalize_event_data(event)
            
            # Проверка обязательных полей
            if not normalized_event.get('title'):
                result['errors'].append('Отсутствует обязательное поле: title')
                return result
            
            # Подготавливаем текст для эмбеддинга
            embedding_text = self._prepare_embedding_text(normalized_event)
            
            # Даты в текстовом формате
            dates_text = normalized_event.get('dates_text', [])
            if isinstance(dates_text, list):
                dates_text_str = '; '.join([str(d) for d in dates_text])[:1000]
            else:
                dates_text_str = str(dates_text)[:500]
            
            # Проверяем, существует ли уже событие
            cursor.execute("SELECT id, last_updated FROM events WHERE id = ?", (event_id,))
            existing_event = cursor.fetchone()
            
            current_time = datetime.now().isoformat()
            
            if existing_event:
                # Обновляем существующее событие
                cursor.execute('''
                UPDATE events SET
                    title=?, description=?, category=?, dates_text=?,
                    price_text=?, place_text=?, url=?, city=?,
                    city_name=?, parsed_at=?, is_free=?, age_restriction=?,
                    image_count=?, embedding_text=?, last_updated=?
                WHERE id=?
                ''', (
                    str(normalized_event.get('title', ''))[:500],
                    str(normalized_event.get('description', ''))[:2000],
                    str(normalized_event.get('category', ''))[:100],
                    dates_text_str,
                    str(normalized_event.get('price_text', ''))[:500],
                    str(normalized_event.get('place_text', ''))[:1000],
                    str(normalized_event.get('url', ''))[:500],
                    str(normalized_event.get('city', ''))[:20],
                    str(normalized_event.get('city_name', ''))[:100],
                    str(normalized_event.get('parsed_at', current_time)),
                    1 if normalized_event.get('is_free') else 0,
                    str(normalized_event.get('age_restriction', '0+'))[:50],  # Исправленное поле
                    normalized_event.get('image_count', 0),
                    embedding_text[:3000],
                    current_time,
                    event_id
                ))
                result['action'] = 'updated'
            else:
                # Вставляем новое событие
                cursor.execute('''
                INSERT INTO events (
                    id, title, description, category, dates_text,
                    price_text, place_text, url, city,
                    city_name, parsed_at, is_free, age_restriction,
                    image_count, embedding_text, last_updated
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    event_id,
                    str(normalized_event.get('title', ''))[:500],
                    str(normalized_event.get('description', ''))[:2000],
                    str(normalized_event.get('category', ''))[:100],
                    dates_text_str,
                    str(normalized_event.get('price_text', ''))[:500],
                    str(normalized_event.get('place_text', ''))[:1000],
                    str(normalized_event.get('url', ''))[:500],
                    str(normalized_event.get('city', ''))[:20],
                    str(normalized_event.get('city_name', ''))[:100],
                    str(normalized_event.get('parsed_at', current_time)),
                    1 if normalized_event.get('is_free') else 0,
                    str(normalized_event.get('age_restriction', '0+'))[:50],  # Исправленное поле
                    normalized_event.get('image_count', 0),
                    embedding_text[:3000],
                    current_time
                ))
                result['action'] = 'created'
            
            # Создаем и сохраняем эмбеддинг
            embedding = self.create_embedding(embedding_text)
            embedding_blob = embedding.astype(np.float32).tobytes()
            
            cursor.execute('''
            INSERT OR REPLACE INTO event_embeddings (event_id, embedding)
            VALUES (?, ?)
            ''', (event_id, embedding_blob))
            
            # Сохраняем детали в JSON
            def safe_json(data):
                try:
                    return json.dumps(data, ensure_ascii=False) if data else '{}'
                except:
                    return '{}'
            
            cursor.execute('''
            INSERT OR REPLACE INTO event_details (
                event_id, dates_json, price_json, place_json,
                tags_json, images_json, participants_json, raw_data_json
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                event_id,
                safe_json(normalized_event.get('dates')),
                safe_json(normalized_event.get('price')),
                safe_json(normalized_event.get('place')),
                safe_json(normalized_event.get('tags')),
                safe_json(normalized_event.get('images')),
                safe_json(normalized_event.get('participants')),
                safe_json(event)  # Сохраняем исходные данные
            ))
            
            self.conn.commit()
            result['success'] = True
            
        except Exception as e:
            self.conn.rollback()
            result['errors'].append(f"Ошибка сохранения: {str(e)}")
            
            # Логируем ошибку в таблицу errors
            try:
                cursor = self.conn.cursor()
                cursor.execute('''
                INSERT INTO processing_errors (event_id, error_type, error_message, raw_data)
                VALUES (?, ?, ?, ?)
                ''', (
                    event.get('id'),
                    'save_error',
                    str(e),
                    json.dumps(event, ensure_ascii=False, default=str)[:1000]
                ))
                self.conn.commit()
            except Exception as log_error:
                logger.error(f"Ошибка логирования ошибки: {log_error}")
        
        return result
    
    def _normalize_event_data(self, event: Dict) -> Dict:
        """Нормализация данных события перед сохранением"""
        normalized = event.copy()
        
        # Нормализация age_restriction
        if 'age_restriction' in normalized:
            normalized['age_restriction'] = validate_age_restriction(normalized['age_restriction'])
        
        # Нормализация is_free
        if 'is_free' in normalized:
            is_free = normalized['is_free']
            if isinstance(is_free, str):
                normalized['is_free'] = is_free.lower() in ['true', '1', 'yes', 'да']
            elif isinstance(is_free, int):
                normalized['is_free'] = bool(is_free)
        
        # Убедимся, что есть обязательные поля
        if 'age_restriction' not in normalized or not normalized['age_restriction']:
            normalized['age_restriction'] = '0+'
        
        return normalized
    
    def _prepare_embedding_text(self, event: Dict) -> str:
        """Подготовка текста для эмбеддинга"""
        embedding_text_parts = []
        
        if event.get('title'):
            embedding_text_parts.append(str(event['title']))
        
        if event.get('description'):
            embedding_text_parts.append(str(event['description'])[:1000])
        
        if event.get('category'):
            embedding_text_parts.append(str(event['category']))
        
        if event.get('tags'):
            tags = event['tags']
            if isinstance(tags, list):
                tag_names = []
                for tag in tags[:10]:
                    if isinstance(tag, dict):
                        tag_names.append(tag.get('name', ''))
                    elif isinstance(tag, str):
                        tag_names.append(tag)
                if tag_names:
                    embedding_text_parts.append(' '.join(tag_names))
        
        return " ".join(embedding_text_parts)
    
    def save_events_batch(self, events: List[Dict]) -> Dict[str, Any]:
        """Пакетное сохранение событий"""
        results = {
            'total': len(events),
            'success': 0,
            'failed': 0,
            'created': 0,
            'updated': 0,
            'errors': [],
            'warnings': []
        }
        
        print(f"\n Сохранение {len(events)} реальных событий в БД...")
        
        for i, event in enumerate(events, 1):
            save_result = self.save_event_with_error_handling(event)
            
            if save_result['success']:
                results['success'] += 1
                if save_result.get('action') == 'created':
                    results['created'] += 1
                else:
                    results['updated'] += 1
            else:
                results['failed'] += 1
                results['errors'].extend(save_result['errors'])
            
            if save_result.get('warnings'):
                results['warnings'].extend(save_result['warnings'])
            
            if i % 20 == 0:
                print(f"  Обработано: {i}/{len(events)}")
        
        print(f" Успешно сохранено: {results['success']} событий")
        print(f" Создано: {results['created']}, Обновлено: {results['updated']}")
        print(f" Ошибок: {results['failed']}")
        
        if results['errors']:
            print(f"  Всего ошибок: {len(results['errors'])}")
        
        return results
    
    def search_similar(self, query: str, limit: int = 10, category: str = None, city: str = None) -> List[Dict]:
        """Поиск похожих событий"""
        try:
            # Создаем эмбеддинг для запроса
            query_embedding = self.create_embedding(query)
            
            cursor = self.conn.cursor()
            
            # Строим запрос с фильтрами
            where_clauses = []
            params = []
            
            if category:
                where_clauses.append("e.category = ?")
                params.append(category)
            
            if city:
                where_clauses.append("e.city = ?")
                params.append(city)
            
            where_sql = " AND ".join(where_clauses) if where_clauses else "1=1"
            
            # Получаем все эмбеддинги
            sql = f'''
            SELECT e.id, e.title, e.category, e.city, emb.embedding
            FROM events e
            JOIN event_embeddings emb ON e.id = emb.event_id
            WHERE {where_sql}
            '''
            
            cursor.execute(sql, params)
            rows = cursor.fetchall()
            
            # Вычисляем косинусное сходство
            results = []
            for event_id, title, category, city, embedding_blob in rows:
                event_embedding = np.frombuffer(embedding_blob, dtype=np.float32)
                
                similarity = np.dot(query_embedding, event_embedding) / (
                    np.linalg.norm(query_embedding) * np.linalg.norm(event_embedding) + 1e-10
                )
                
                results.append({
                    'event_id': event_id,
                    'title': title,
                    'category': category,
                    'city': city,
                    'similarity': float(similarity)
                })
            
            # Сортируем по сходству
            results.sort(key=lambda x: x['similarity'], reverse=True)
            
            # Получаем полную информацию для топ-результатов
            for result in results[:limit]:
                cursor.execute('''
                SELECT description, dates_text, price_text, place_text, url, age_restriction, is_free
                FROM events WHERE id = ?
                ''', (result['event_id'],))
                
                row = cursor.fetchone()
                if row:
                    result.update({
                        'description': row[0],
                        'dates_text': row[1],
                        'price_text': row[2],
                        'place_text': row[3],
                        'url': row[4],
                        'age_restriction': row[5] or '0+',
                        'is_free': bool(row[6])
                    })
            
            return results[:limit]
            
        except Exception as e:
            logger.error(f"Ошибка поиска: {e}")
            return []
    
    def get_stats(self) -> Dict:
        """Статистика базы данных"""
        try:
            cursor = self.conn.cursor()
            
            stats = {
                'total_events': 0,
                'by_category': {},
                'by_city': {},
                'with_images': 0,
                'free_events': 0,
                'recent_events': 0,
                'age_restrictions': {},
                'processing_errors': 0
            }
            
            # Общее количество
            cursor.execute("SELECT COUNT(*) FROM events")
            stats['total_events'] = cursor.fetchone()[0] or 0
            
            # По категориям
            cursor.execute("SELECT category, COUNT(*) FROM events GROUP BY category ORDER BY COUNT(*) DESC")
            stats['by_category'] = {row[0]: row[1] for row in cursor.fetchall()}
            
            # По городам
            cursor.execute("SELECT city, COUNT(*) FROM events GROUP BY city")
            stats['by_city'] = {row[0]: row[1] for row in cursor.fetchall()}
            
            # С изображениями
            cursor.execute("SELECT COUNT(*) FROM events WHERE image_count > 0")
            stats['with_images'] = cursor.fetchone()[0] or 0
            
            # Бесплатные
            cursor.execute("SELECT COUNT(*) FROM events WHERE is_free = 1")
            stats['free_events'] = cursor.fetchone()[0] or 0
            
            # Недавние (за последние 7 дней)
            week_ago = (datetime.now() - timedelta(days=7)).isoformat()
            cursor.execute("SELECT COUNT(*) FROM events WHERE parsed_at > ?", (week_ago,))
            stats['recent_events'] = cursor.fetchone()[0] or 0
            
            # Возрастные ограничения
            cursor.execute("SELECT age_restriction, COUNT(*) FROM events WHERE age_restriction IS NOT NULL GROUP BY age_restriction ORDER BY COUNT(*) DESC LIMIT 10")
            stats['age_restrictions'] = {row[0]: row[1] for row in cursor.fetchall()}
            
            # Ошибки обработки
            cursor.execute("SELECT COUNT(*) FROM processing_errors")
            stats['processing_errors'] = cursor.fetchone()[0] or 0
            
            return stats
            
        except Exception as e:
            logger.error(f"Ошибка получения статистики: {e}")
            return {}
    
    def fix_existing_age_restrictions(self):
        """Исправляет существующие записи с некорректными age_restriction"""
        try:
            cursor = self.conn.cursor()
            
            # Находим события с проблемными age_restriction
            cursor.execute("""
            SELECT id, age_restriction FROM events 
            WHERE age_restriction IS NULL 
               OR age_restriction = '0' 
               OR age_restriction = ''
               OR age_restriction = 'null'
               OR age_restriction = 'undefined'
            """)
            
            problematic = cursor.fetchall()
            fixed_count = 0
            
            print(f"Найдено проблемных записей: {len(problematic)}")
            
            for event_id, age in problematic:
                # Нормализуем age_restriction
                normalized_age = validate_age_restriction(age)
                if not normalized_age:
                    normalized_age = "0+"
                
                cursor.execute(
                    "UPDATE events SET age_restriction = ? WHERE id = ?",
                    (normalized_age, event_id)
                )
                fixed_count += 1
            
            self.conn.commit()
            print(f"Исправлено записей: {fixed_count}")
            
            return fixed_count
            
        except Exception as e:
            logger.error(f"Ошибка исправления age_restriction: {e}")
            return 0
    
    def close(self):
        """Закрытие соединения"""
        if self.conn:
            self.conn.close()

print("  Векторная БД для реальных данных создана с исправленной схемой!")

# ============ 5. ИСПРАВЛЕННЫЙ ПАЙПЛАЙН ДЛЯ РЕАЛЬНЫХ ДАННЫХ ============
class RealDataPipeline:
    """Полный пайплайн для реальных данных с исправленной обработкой"""
    
    def __init__(self, city: str = "msk"):
        self.city = city
        self.city_name = "Москва" if city == "msk" else "Санкт-Петербург"
        
        # Инициализируем парсер
        self.parser = RealKudaGoParser(
            city=city,
            max_events_per_category=100,
            page_size=100,
            enable_enrichment=True,
            days_ahead=365
        )
        
        # Инициализируем БД
        self.vector_db = RealEventVectorDatabase(
            db_path=f"real_events_{city}_vector.db"
        )
        
        self.parsed_events = None
        self.pydantic_models = None
        
        print(f" Пайплайн для реальных данных инициализирован!")
        print(f" Город: {self.city_name}")
    
    async def run_pipeline(self) -> Dict[str, Any]:
        """Запуск полного пайплайна"""
        print("\n" + "="*70)
        print(f" ЗАПУСК ПАЙПЛАЙНА ДЛЯ {self.city_name}")
        print("="*70)
        
        # 1. Парсинг реальных данных
        print("\n1.  ПАРСИНГ РЕАЛЬНЫХ ДАННЫХ С KUDAGO")
        print("-"*40)
        
        events_by_category = await self.parser.parse_all_categories(max_concurrent=3)
        
        # Объединяем все события
        all_events = []
        for category, events in events_by_category.items():
            all_events.extend(events)
        
        print(f"\n Всего спаршено реальных событий: {len(all_events)}")
        
        if not all_events:
            print(" Нет данных для обработки")
            return None
        
        # 2. Конвертация в Pydantic модели
        print("\n2.  КОНВЕРТАЦИЯ В PYDANTIC МОДЕЛИ")
        print("-"*40)
        
        self.pydantic_models = self.convert_to_pydantic(all_events)
        print(f" Конвертировано моделей: {len(self.pydantic_models)}")
        
        # 3. Сохранение в JSON
        print("\n3.  СОХРАНЕНИЕ В JSON")
        print("-"*40)
        
        json_path = self.save_to_json(all_events)
        
        # 4. Исправление существующих данных в БД
        print("\n4.  ИСПРАВЛЕНИЕ СУЩЕСТВУЮЩИХ ДАННЫХ")
        print("-"*40)
        
        fixed_count = self.vector_db.fix_existing_age_restrictions()
        if fixed_count > 0:
            print(f" Исправлено существующих записей: {fixed_count}")
        
        # 5. Загрузка в векторную БД
        print("\n5.  ЗАГРУЗКА В ВЕКТОРНУЮ БД")
        print("-"*40)
        
        save_result = self.vector_db.save_events_batch(all_events)
        print(f" Успешно сохранено в БД: {save_result['success']} событий")
        
        # 6. Статистика
        print("\n6.  СТАТИСТИКА РЕАЛЬНЫХ ДАННЫХ")
        print("-"*40)
        
        self.show_statistics(all_events)
        
        # 7. Пример поиска
        print("\n7.  ПРИМЕРЫ ПОИСКА ПО РЕАЛЬНЫМ ДАННЫМ")
        print("-"*40)
        
        self.example_search()
        
        # Итоги
        print("\n" + "="*70)
        print(" ПАЙПЛАЙН УСПЕШНО ЗАВЕРШЕН!")
        print("="*70)
        
        return {
            'city': self.city,
            'city_name': self.city_name,
            'total_events': len(all_events),
            'pydantic_models': len(self.pydantic_models),
            'json_file': json_path,
            'db_stats': self.vector_db.get_stats(),
            'save_result': save_result,
            'fixed_existing': fixed_count
        }
    
    def convert_to_pydantic(self, events: List[Dict]) -> List[EventModel]:
        """Конвертация в Pydantic модели с обработкой ошибок"""
        pydantic_events = []
        converted = 0
        errors = 0
        error_details = []
        
        print(f"Конвертация {len(events)} реальных событий...")
        
        for event_dict in events:
            try:
                # Предобработка данных
                self._preprocess_event_data(event_dict)
                
                # Определяем категорию
                category = self._determine_category(event_dict)
                
                # Создаем модель события
                event_model = EventModel(
                    id=event_dict['id'],
                    title=event_dict['title'],
                    short_title=event_dict.get('short_title'),
                    description=event_dict.get('description'),
                    slug=event_dict.get('slug'),
                    category=category,
                    tags=self._prepare_tags(event_dict),
                    dates=self._prepare_dates(event_dict),
                    age_restriction=event_dict.get('age_restriction'),
                    place=self._prepare_place(event_dict),
                    place_id=event_dict.get('place_id'),
                    price=self._prepare_price(event_dict),
                    images=self._prepare_images(event_dict),
                    participants=self._prepare_participants(event_dict),
                    url=event_dict.get('url', ''),
                    site_url=event_dict.get('site_url'),
                    buy_url=event_dict.get('buy_url'),
                    favorites_count=event_dict.get('favorites_count', 0),
                    comments_count=event_dict.get('comments_count', 0),
                    location=event_dict.get('city', self.city),
                    is_free=event_dict.get('is_free', False),
                    publication_date=event_dict.get('publication_date')
                )
                
                pydantic_events.append(event_model)
                converted += 1
                
                if converted % 20 == 0:
                    print(f"  Конвертировано: {converted}/{len(events)}")
                    
            except Exception as e:
                errors += 1
                error_details.append({
                    'event_id': event_dict.get('id'),
                    'title': event_dict.get('title', 'No title'),
                    'error': str(e),
                    'data': {k: v for k, v in event_dict.items() if k not in ['description', 'place', 'images']}
                })
                logger.debug(f"Ошибка конвертации события {event_dict.get('id')}: {e}")
        
        print(f" Успешно: {converted},  Ошибок: {errors}")
        
        # Сохраняем детали ошибок
        if error_details:
            error_file = Path(f"real_events_data/conversion_errors_{self.city}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
            with open(error_file, 'w', encoding='utf-8') as f:
                json.dump(error_details, f, ensure_ascii=False, indent=2, default=str)
            print(f"  Детали ошибок сохранены в: {error_file}")
        
        return pydantic_events
    
    def _preprocess_event_data(self, event_dict: Dict):
        """Предобработка данных события"""
        # Очистка текстовых полей
        for field in ['title', 'description', 'age_restriction', 'price_text', 'place_text']:
            if field in event_dict and event_dict[field] is not None:
                event_dict[field] = str(event_dict[field]).strip()
        
        # Убедимся, что есть ID
        if 'id' not in event_dict or not event_dict['id']:
            event_dict['id'] = int(datetime.now().timestamp() * 1000) + hash(str(event_dict.get('title', '')))
        
        # Убедимся, что есть URL
        if 'url' not in event_dict or not event_dict['url']:
            event_dict['url'] = f"https://kudago.com/{self.city}/event/{event_dict['id']}/"
        
        # Нормализация age_restriction
        if 'age_restriction' in event_dict:
            event_dict['age_restriction'] = validate_age_restriction(event_dict['age_restriction'])
        else:
            event_dict['age_restriction'] = "0+"
        
        # Нормализация is_free
        if 'is_free' in event_dict:
            is_free = event_dict['is_free']
            if isinstance(is_free, str):
                event_dict['is_free'] = is_free.lower() in ['true', '1', 'yes', 'да']
            elif isinstance(is_free, int):
                event_dict['is_free'] = bool(is_free)
    
    def _determine_category(self, event_dict: Dict) -> EventCategory:
        """Определение категории события"""
        category_str = event_dict.get('category', '').lower()
        
        # Расширенное сопоставление категорий
        category_mapping = {
            'концерт': EventCategory.CONCERT,
            'концерты': EventCategory.CONCERT,
            'concert': EventCategory.CONCERT,
            'музыка': EventCategory.CONCERT,
            
            'театр': EventCategory.THEATER,
            'theater': EventCategory.THEATER,
            'theatre': EventCategory.THEATER,
            'спектакль': EventCategory.THEATER,
            
            'выставка': EventCategory.EXHIBITION,
            'выставки': EventCategory.EXHIBITION,
            'exhibition': EventCategory.EXHIBITION,
            'expo': EventCategory.EXHIBITION,
            
            'фестиваль': EventCategory.FESTIVAL,
            'фестивали': EventCategory.FESTIVAL,
            'festival': EventCategory.FESTIVAL,
            
            'образование': EventCategory.EDUCATION,
            'education': EventCategory.EDUCATION,
            'обучение': EventCategory.EDUCATION,
            
            'лекция': EventCategory.LECTURE,
            'lecture': EventCategory.LECTURE,
            'семинар': EventCategory.LECTURE,
            'конференция': EventCategory.LECTURE,
            
            'вечеринка': EventCategory.PARTY,
            'вечеринки': EventCategory.PARTY,
            'party': EventCategory.PARTY,
            'ночной клуб': EventCategory.PARTY,
            
            'спорт': EventCategory.SPORT,
            'sport': EventCategory.SPORT,
            'спортивный': EventCategory.SPORT,
            
            'квест': EventCategory.QUEST,
            'квесты': EventCategory.QUEST,
            'quest': EventCategory.QUEST,
            
            'экскурсия': EventCategory.EXCURSION,
            'экскурсии': EventCategory.EXCURSION,
            'excursion': EventCategory.EXCURSION,
            'тур': EventCategory.TOUR,
            'tour': EventCategory.TOUR,
            
            'шоу': EventCategory.SHOW,
            'show': EventCategory.SHOW,
            
            'стендап': EventCategory.STANDUP,
            'standup': EventCategory.STANDUP,
            'stand-up': EventCategory.STANDUP,
            'юмор': EventCategory.STANDUP,
            
            'детск': EventCategory.KIDS,
            'kids': EventCategory.KIDS,
            'children': EventCategory.KIDS,
            'ребенок': EventCategory.KIDS,
            
            'мода': EventCategory.FASHION,
            'fashion': EventCategory.FASHION,
            
            'гастроном': EventCategory.GASTRONOMY,
            'gastronomy': EventCategory.GASTRONOMY,
            'еда': EventCategory.GASTRONOMY,
            'food': EventCategory.GASTRONOMY,
            'ресторан': EventCategory.GASTRONOMY,
            
            'кино': EventCategory.CINEMA,
            'cinema': EventCategory.CINEMA,
            'фильм': EventCategory.CINEMA,
            'movie': EventCategory.CINEMA,
            
            'мастер-класс': EventCategory.MASTERCLASS,
            'masterclass': EventCategory.MASTERCLASS,
            'workshop': EventCategory.MASTERCLASS,
            'мастеркласс': EventCategory.MASTERCLASS,
            
            'опера': EventCategory.SHOW,
            'opera': EventCategory.SHOW,
            
            'балет': EventCategory.SHOW,
            'ballet': EventCategory.SHOW,
            
            'цирк': EventCategory.SHOW,
            'circus': EventCategory.SHOW,
            
            'ярмарка': EventCategory.FESTIVAL,
            'fair': EventCategory.FESTIVAL,
        }
        
        # Поиск категории по ключевым словам
        for key, value in category_mapping.items():
            if key in category_str:
                return value
        
        # Попробуем определить по тегам
        if 'tags' in event_dict:
            for tag in event_dict['tags']:
                tag_name = tag.get('name', '').lower() if isinstance(tag, dict) else str(tag).lower()
                for key, value in category_mapping.items():
                    if key in tag_name:
                        return value
        
        return EventCategory.OTHER
    
    def _prepare_tags(self, event_dict: Dict) -> List[TagModel]:
        """Подготовка тегов"""
        tags = []
        raw_tags = event_dict.get('tags', [])
        
        if not raw_tags:
            # Если нет тегов, создаем один тег из категории
            category = event_dict.get('category', 'другое')
            tags.append(TagModel(name=category))
            return tags
        
        for tag in raw_tags[:10]:
            if isinstance(tag, dict):
                name = tag.get('name', '')
                if name:
                    tags.append(TagModel(
                        name=str(name)[:100],
                        slug=tag.get('slug'),
                        id=tag.get('id')
                    ))
            elif isinstance(tag, str) and tag:
                tags.append(TagModel(name=str(tag)[:100]))
        
        return tags
    
    def _prepare_dates(self, event_dict: Dict) -> List[DateModel]:
        """Подготовка дат"""
        dates = []
        raw_dates = event_dict.get('dates', [])
        
        if not raw_dates:
            # Если нет дат, создаем дефолтную дату (завтра)
            tomorrow = datetime.now() + timedelta(days=1)
            dates.append(DateModel(
                start=int(tomorrow.timestamp()),
                end=int((tomorrow + timedelta(hours=2)).timestamp())
            ))
            return dates
        
        for date_dict in raw_dates:
            if isinstance(date_dict, dict) and date_dict.get('start'):
                try:
                    start = date_dict.get('start')
                    if not isinstance(start, (int, float)):
                        continue
                    
                    end = date_dict.get('end')
                    if end and not isinstance(end, (int, float)):
                        end = None
                    
                    dates.append(DateModel(
                        start=int(start),
                        end=int(end) if end else None,
                        is_continuous=date_dict.get('is_continuous', False)
                    ))
                except Exception as e:
                    logger.debug(f"Ошибка обработки даты: {e}")
                    continue
        
        return dates
    
    def _prepare_place(self, event_dict: Dict) -> Optional[PlaceModel]:
        """Подготовка информации о месте"""
        place_dict = event_dict.get('place')
        
        if not place_dict or not isinstance(place_dict, dict):
            return None
        
        try:
            # Извлекаем и валидируем координаты
            coords = place_dict.get('coords')
            if coords and isinstance(coords, dict):
                lat = coords.get('lat')
                lon = coords.get('lon')
                if lat is not None and lon is not None:
                    try:
                        lat = float(lat)
                        lon = float(lon)
                        coords = {'lat': lat, 'lon': lon}
                    except (ValueError, TypeError):
                        coords = None
                else:
                    coords = None
            
            return PlaceModel(
                id=place_dict.get('id'),
                title=str(place_dict.get('title', 'Неизвестно'))[:200],
                address=str(place_dict.get('address', ''))[:500],
                subway=place_dict.get('subway'),
                coords=coords,
                phone=place_dict.get('phone'),
                site_url=place_dict.get('site_url'),
                city=place_dict.get('city'),
                is_closed=place_dict.get('is_closed', False),
                working_hours=place_dict.get('working_hours')
            )
        except Exception as e:
            logger.debug(f"Ошибка создания PlaceModel: {e}")
            return None
    
    def _prepare_price(self, event_dict: Dict) -> Optional[PriceModel]:
        """Подготовка информации о цене"""
        price_dict = event_dict.get('price')
        
        if not price_dict or not isinstance(price_dict, dict):
            # Проверяем поле is_free
            is_free = event_dict.get('is_free', False)
            if is_free:
                return PriceModel(is_free=True)
            return None
        
        try:
            # Извлекаем и валидируем числовые значения
            min_price = price_dict.get('min')
            max_price = price_dict.get('max')
            
            if min_price is not None:
                try:
                    min_price = float(min_price)
                except (ValueError, TypeError):
                    min_price = None
            
            if max_price is not None:
                try:
                    max_price = float(max_price)
                except (ValueError, TypeError):
                    max_price = None
            
            return PriceModel(
                is_free=price_dict.get('is_free', False),
                min=min_price,
                max=max_price,
                currency=price_dict.get('currency', 'RUB'),
                description=price_dict.get('description')
            )
        except Exception as e:
            logger.debug(f"Ошибка создания PriceModel: {e}")
            return None
    
    def _prepare_images(self, event_dict: Dict) -> List[ImageModel]:
        """Подготовка изображений"""
        images = []
        raw_images = event_dict.get('images', [])
        
        for img_dict in raw_images[:5]:
            if isinstance(img_dict, dict) and img_dict.get('url'):
                try:
                    images.append(ImageModel(
                        url=str(img_dict.get('url', '')),
                        thumbnail=str(img_dict.get('thumbnail', '')) if img_dict.get('thumbnail') else None,
                        source=img_dict.get('source')
                    ))
                except Exception as e:
                    logger.debug(f"Ошибка создания ImageModel: {e}")
                    continue
        
        return images
    
    def _prepare_participants(self, event_dict: Dict) -> List[ParticipantModel]:
        """Подготовка информации об участниках"""
        participants = []
        raw_participants = event_dict.get('participants', [])
        
        for part_dict in raw_participants[:5]:
            if isinstance(part_dict, dict):
                name = part_dict.get('name') or part_dict.get('title', '')
                if name:
                    participants.append(ParticipantModel(
                        name=str(name)[:200],
                        role=part_dict.get('role'),
                        images=part_dict.get('images')
                    ))
        
        return participants
    
    def save_to_json(self, events: List[Dict]) -> str:
        """Сохранение в JSON файл"""
        # Создаем директорию для данных
        data_dir = Path("real_events_data")
        data_dir.mkdir(exist_ok=True)
        
        # Генерируем имя файла
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"events_{self.city}_{timestamp}.json"
        filepath = data_dir / filename
        
        # Сохраняем
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(events, f, ensure_ascii=False, indent=2, default=str)
        
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        
        print(f" Файл сохранен: {filepath}")
        print(f" Размер: {size_mb:.2f} MB")
        print(f" Событий: {len(events)}")
        print(f" Город: {self.city_name}")
        
        # Сохраняем Pydantic модели отдельно
        if self.pydantic_models:
            pydantic_file = data_dir / f"events_pydantic_{self.city}_{timestamp}.json"
            pydantic_data = [event.model_dump() for event in self.pydantic_models]
            
            with open(pydantic_file, 'w', encoding='utf-8') as f:
                json.dump(pydantic_data, f, ensure_ascii=False, indent=2, default=str)
            
            print(f" Pydantic модели сохранены в: {pydantic_file}")
        
        return str(filepath)
    
    def show_statistics(self, events: List[Dict]):
        """Отображение статистики реальных данных"""
        if not events:
            print(" Нет данных для статистики")
            return
        
        print(f" СТАТИСТИКА РЕАЛЬНЫХ СОБЫТИЙ:")
        print(f"   Всего событий: {len(events)}")
        
        # По категориям
        categories = {}
        for event in events:
            cat = event.get('category', 'Неизвестно')
            categories[cat] = categories.get(cat, 0) + 1
        
        print(f"   Категорий: {len(categories)}")
        print(f"   Топ категорий:")
        for cat, count in sorted(categories.items(), key=lambda x: x[1], reverse=True)[:10]:
            percentage = (count / len(events)) * 100
            print(f"     • {cat}: {count} ({percentage:.1f}%)")
        
        # С датами
        events_with_dates = sum(1 for e in events if e.get('dates'))
        print(f"   Событий с указанием дат: {events_with_dates} ({events_with_dates/len(events)*100:.1f}%)")
        
        # С местами
        events_with_place = sum(1 for e in events if e.get('place'))
        print(f"   Событий с указанием места: {events_with_place} ({events_with_place/len(events)*100:.1f}%)")
        
        # Бесплатные
        free_events = sum(1 for e in events if e.get('is_free'))
        print(f"   Бесплатных событий: {free_events} ({free_events/len(events)*100:.1f}%)")
        
        # С изображениями
        events_with_images = sum(1 for e in events if e.get('images'))
        print(f"   Событий с изображениями: {events_with_images} ({events_with_images/len(events)*100:.1f}%)")
        
        # С описанием
        events_with_description = sum(1 for e in events if e.get('description'))
        print(f"   Событий с описанием: {events_with_description} ({events_with_description/len(events)*100:.1f}%)")
        
        # С тегами
        events_with_tags = sum(1 for e in events if e.get('tags'))
        print(f"   Событий с тегами: {events_with_tags} ({events_with_tags/len(events)*100:.1f}%)")
        
        # Возрастные ограничения
        age_stats = {}
        for event in events:
            age = event.get('age_restriction', '0+')
            age_stats[age] = age_stats.get(age, 0) + 1
        
        print(f"   Возрастные ограничения:")
        for age, count in sorted(age_stats.items(), key=lambda x: x[1], reverse=True)[:5]:
            percentage = (count / len(events)) * 100
            print(f"     • {age}: {count} ({percentage:.1f}%)")
    
    def example_search(self):
        """Примеры поиска по реальным данным"""
        search_examples = [
            ("концерт музыка", None, "Музыкальный концерт"),
            ("выставка искусство", None, "Художественная выставка"),
            ("детский праздник", "kids", "Детское событие"),
            ("бесплатный мастер-класс", "education", "Бесплатное обучение"),
            ("кино премьера", "cinema", "Кинособытие")
        ]
        
        for query, category, description in search_examples[:3]:
            print(f"\n Поиск: '{query}'")
            results = self.vector_db.search_similar(query, limit=3, city=self.city)
            
            if results:
                print(f"   Найдено {len(results)} похожих событий:")
                for i, result in enumerate(results, 1):
                    print(f"   {i}. {result['title'][:60]}...")
                    print(f"      Категория: {result['category']}")
                    print(f"      Возраст: {result.get('age_restriction', '0+')}")
                    print(f"      Бесплатно: {'Да' if result.get('is_free') else 'Нет'}")
                    print(f"      Сходство: {result['similarity']:.3f}")
                    if result.get('price_text'):
                        print(f"      Цена: {result['price_text']}")
            else:
                print("    Похожих событий не найдено")
    
    def search_events(self, query: str, limit: int = 10, category: str = None):
        """Поиск событий"""
        return self.vector_db.search_similar(query, limit=limit, category=category, city=self.city)
    
    def get_database_stats(self):
        """Статистика БД"""
        return self.vector_db.get_stats()
    
    def close(self):
        """Закрытие соединений"""
        self.vector_db.close()

print("  Пайплайн для реальных данных создан с исправленной обработкой!")

# ============ 6. ФУНКЦИИ ДЛЯ ЗАПУСКА ============
async def parse_moscow_real():
    """Парсинг реальных событий Москвы"""
    print("\n" + "="*70)
    print(" ПАРСИНГ РЕАЛЬНЫХ СОБЫТИЙ МОСКВЫ")
    print("="*70)
    
    pipeline = RealDataPipeline(city="msk")
    results = await pipeline.run_pipeline()
    
    if results:
        print(f"\n РЕЗУЛЬТАТЫ МОСКВЫ:")
        print(f"   Город: {results['city_name']}")
        print(f"   Событий: {results['total_events']}")
        print(f"   Pydantic моделей: {results['pydantic_models']}")
        print(f"   Исправлено существующих записей: {results.get('fixed_existing', 0)}")
        print(f"   JSON файл: {results['json_file']}")
        
        stats = results['db_stats']
        print(f"    Статистика БД:")
        print(f"     • Всего событий: {stats.get('total_events', 0)}")
        print(f"     • Категорий: {len(stats.get('by_category', {}))}")
        print(f"     • С изображениями: {stats.get('with_images', 0)}")
        print(f"     • Бесплатных: {stats.get('free_events', 0)}")
        print(f"     • Возрастные ограничения: {len(stats.get('age_restrictions', {}))} различных")
    
    return results

async def parse_spb_real():
    """Парсинг реальных событий Санкт-Петербурга"""
    print("\n" + "="*70)
    print(" ПАРСИНГ РЕАЛЬНЫХ СОБЫТИЙ САНКТ-ПЕТЕРБУРГА")
    print("="*70)
    
    pipeline = RealDataPipeline(city="spb")
    results = await pipeline.run_pipeline()
    
    if results:
        print(f"\n РЕЗУЛЬТАТЫ САНКТ-ПЕТЕРБУРГА:")
        print(f"   Город: {results['city_name']}")
        print(f"   Событий: {results['total_events']}")
        print(f"   Pydantic моделей: {results['pydantic_models']}")
        print(f"   Исправлено существующих записей: {results.get('fixed_existing', 0)}")
        print(f"   JSON файл: {results['json_file']}")
    
    return results

async def parse_both_cities_real():
    """Парсинг реальных событий обоих городов"""
    print("\n" + "="*80)
    print(" ПАРСИНГ РЕАЛЬНЫХ СОБЫТИЙ МОСКВЫ И САНКТ-ПЕТЕРБУРГА")
    print("="*80)
    
    results = {}
    
    # Москва
    print("\n1. МОСКВА")
    print("-"*40)
    results["msk"] = await parse_moscow_real()
    
    # Пауза между городами
    print("\n Пауза 10 секунд...")
    await asyncio.sleep(10)
    
    # Санкт-Петербург
    print("\n2. САНКТ-ПЕТЕРБУРГ")
    print("-"*40)
    results["spb"] = await parse_spb_real()
    
    # Итоги
    print("\n" + "="*70)
    print(" ИТОГИ ПО ОБОИМ ГОРОДАМ")
    print("="*70)
    
    total_events = 0
    total_pydantic = 0
    total_fixed = 0
    
    for city, result in results.items():
        if result:
            events = result.get('total_events', 0)
            pydantic = result.get('pydantic_models', 0)
            fixed = result.get('fixed_existing', 0)
            total_events += events
            total_pydantic += pydantic
            total_fixed += fixed
            
            city_name = "Москва" if city == "msk" else "Санкт-Петербург"
            print(f"\n{city_name}:")
            print(f"  • Событий: {events}")
            print(f"  • Pydantic моделей: {pydantic}")
            print(f"  • Исправлено записей: {fixed}")
            if result.get('json_file'):
                print(f"  • JSON файл: {os.path.basename(result['json_file'])}")
            
            # Категории
            db_stats = result.get('db_stats', {})
            categories = db_stats.get('by_category', {})
            if categories:
                print(f"  • Топ-3 категории:")
                for cat, count in list(categories.items())[:3]:
                    print(f"    - {cat}: {count}")
    
    print(f"\n ВСЕГО:")
    print(f"  • Событий: {total_events}")
    print(f"  • Pydantic моделей: {total_pydantic}")
    print(f"  • Исправлено записей: {total_fixed}")
    print(f"  • Данные: реальные с Kudago API")
    print(f"  • Обработка ошибок: включена")
    print(f"  • Валидация данных: исправлена")
    
    return results

# Создаем папку для данных
Path("real_events_data").mkdir(exist_ok=True)

# ============ 7. ЗАПУСК ПАЙПЛАЙНА ============
# Для запуска выполните:
# results = await parse_both_cities_real()
# или для одного города:
# results = await parse_moscow_real()
# results = await parse_spb_real()

  Библиотеки загружены и окружение настроено!
  Pydantic модели созданы с исправленной валидацией!
  Реальный парсер Kudago создан с исправленной обработкой данных!
  Векторная БД для реальных данных создана с исправленной схемой!
  Пайплайн для реальных данных создан с исправленной обработкой!


In [2]:
results = await parse_both_cities_real()

2025-12-12 13:06:38,927 - __main__ - INFO - Парсер инициализирован для города: Москва
2025-12-12 13:06:38,927 - __main__ - INFO - Категорий: 12
2025-12-12 13:06:38,928 - __main__ - INFO - Цель: 100 событий на категорию



 ПАРСИНГ РЕАЛЬНЫХ СОБЫТИЙ МОСКВЫ И САНКТ-ПЕТЕРБУРГА

1. МОСКВА
----------------------------------------

 ПАРСИНГ РЕАЛЬНЫХ СОБЫТИЙ МОСКВЫ


2025-12-12 13:06:42,417 - __main__ - INFO -   Страница 1...
2025-12-12 13:06:42,418 - __main__ - INFO -   Страница 1...
2025-12-12 13:06:42,420 - __main__ - INFO -   Страница 1...


 Векторная БД создана: real_events_msk_vector.db
 Размерность эмбеддингов: 384
 Пайплайн для реальных данных инициализирован!
 Город: Москва

 ЗАПУСК ПАЙПЛАЙНА ДЛЯ Москва

1.  ПАРСИНГ РЕАЛЬНЫХ ДАННЫХ С KUDAGO
----------------------------------------

 Начинаем парсинг города: Москва
 Категорий: 12
 Цель: 100 событий на категорию
--------------------------------------------------

 Парсинг категории: Концерты

 Парсинг категории: Театр

 Парсинг категории: Выставки


2025-12-12 13:07:55,471 - __main__ - INFO -   Спаршено: 100 событий
2025-12-12 13:07:56,485 - __main__ - INFO -   Страница 1...


...

   Категория 'Стендап': 0 событий


2025-12-12 13:11:37,029 - __main__ - INFO -   Спаршено: 81 событий


   Категория 'Детские': 81 событий


2025-12-12 13:11:50,838 - __main__ - INFO -   Спаршено: 99 событий


   Категория 'Образование': 99 событий

ПАРСИНГ ЗАВЕРШЕН!
 Город: Санкт-Петербург
 Всего событий: 518

 СТАТИСТИКА ПО КАТЕГОРИЯМ:
   Концерты: 100 событий
   Театр: 100 событий
   Выставки: 100 событий
   Фестивали: 12 событий
   Образование: 99 событий
   Вечеринки: 17 событий
   Шоу: 0 событий
   Детские: 81 событий
   Кино: 8 событий
   Мода: 1 событий
   Гастрономия: 0 событий
   Стендап: 0 событий

 Успешных категорий: 9/12

 Всего спаршено реальных событий: 518

2.  КОНВЕРТАЦИЯ В PYDANTIC МОДЕЛИ
----------------------------------------
Конвертация 518 реальных событий...
  Конвертировано: 20/518
  Конвертировано: 40/518
  Конвертировано: 60/518
  Конвертировано: 80/518
  Конвертировано: 100/518
  Конвертировано: 120/518
  Конвертировано: 140/518
  Конвертировано: 160/518
  Конвертировано: 180/518
  Конвертировано: 200/518
  Конвертировано: 220/518
  Конвертировано: 240/518
  Конвертировано: 260/518
  Конвертировано: 280/518
  Конвертировано: 300/518
  Конвертировано: 320/518
  Ко